In [1]:
import pandas as pd
from datetime import timedelta

In [2]:
# Read in data
payments = pd.read_csv("../Resources/payments.csv")

## 3) Devise a method of generating a table of all Accounts with their cumulative payments to date as of any arbitrary timestamp in the past.

In [ ]:
# Preview dataframe
payments.head()

In [3]:
# Check for unique currencies
payments.currency.unique()

array(['KES'], dtype=object)

In [4]:
# Set currency
currency = payments.currency[0]

In [5]:
# Variable to take timestamp input.
timestamp_str = input("Enter date with YYYY-MM-DD format.")

Enter date with YYYY-MM-DD format.2020-03-01


In [6]:
# Convert variable from string to datetime object
timestamp_dt = pd.to_datetime(timestamp_str)

# Adjust timestamp to end of day
timestamp_dt = timestamp_dt + timedelta(hours=23,minutes=59,seconds=59)

In [7]:
# Change data type from string to datetime object
payments['effective_when'] = pd.to_datetime(payments['effective_when'])

In [8]:
# Create limited dataframe starting at indicated timestamp 
payments_date_limited = payments.loc[payments['effective_when'] <= timestamp_dt]
payments_date_limited.head()

,id,effective_when,account_id,currency,amount
0,1,2019-12-26 06:00:42.571392,1,KES,950
1,2,2019-12-26 06:00:42.571392,1,KES,950
2,3,2019-11-06 05:49:39.571392,2,KES,950
3,4,2019-11-06 05:49:39.571392,2,KES,950
4,5,2019-11-11 06:43:34.571392,2,KES,415


In [9]:
# Group by account_id and perform aggregate functions
cumulative_payments = payments_date_limited.groupby('account_id', as_index=False).agg({'id':'count','amount':'sum'})

# Rename columns
cumulative_payments = cumulative_payments.rename(columns={
    'id':'total_num_payments',
    'amount':f'total_sum_payments_{currency}'})

# Preview dataframe
print(f"CUMULATIVE PAYMENTS BY ACCOUNT AS OF: {timestamp_str[:10]}")
cumulative_payments.head()

CUMULATIVE PAYMENTS BY ACCOUNT AS OF: 2020-03-01


,account_id,total_num_payments,total_sum_payments_KES
0,1,2,1900
1,2,18,8100
2,3,15,6711
3,4,13,8100
4,5,13,5793


In [19]:
# Export as csv file
cumulative_payments.to_csv(f"../Output/cumulative_payments_by_account_as_of_{timestamp_str[:10]}.csv", index=False)